In [8]:
import os
import itk
import vtk
from vtk import vtkCommand
import numpy as np
import matplotlib.pyplot as plt

In [9]:
# DICOM_PATH = "./001321140"
DICOM_PATH = "/Users/benjaminhon/Developer/data/bpynz/M489106/001321140"

# Read DICOM to Image

In [3]:
# image type pixel type, dimensions
inputImageType = itk.Image[itk.F, 3]
reader = itk.ImageSeriesReader[inputImageType].New()
# generate dicom series file names
generator = itk.GDCMSeriesFileNames.New()
generator.SetDirectory(DICOM_PATH)
seriesUIDs = generator.GetSeriesUIDs()
series = { uid: generator.GetFileNames(uid) for uid in generator.GetSeriesUIDs() }
# itk series reader, select a series of file names
reader.SetFileNames(series[seriesUIDs[3]])

# ITK to VTK

In [4]:
# output image is UC
outputImageType = itk.Image[itk.UC, 3]

# itk series reader 
# -> RescaleIntensityImageFilter 
# -> CastImageFilter 
# -> ImageToVTKImageFilter 
# -> vtk image
rescaler = itk.RescaleIntensityImageFilter[inputImageType, inputImageType].New()
rescaler.SetInput(reader.GetOutput())
rescaler.SetOutputMaximum(255)
rescaler.SetOutputMinimum(0)

caster = itk.CastImageFilter[inputImageType, outputImageType].New()
caster.SetInput(rescaler.GetOutput())

itkToVtkConverter = itk.ImageToVTKImageFilter[outputImageType].New()
itkToVtkConverter.SetInput(caster.GetOutput())
itkToVtkConverter.Update()

imageData = itkToVtkConverter.GetOutput()
print('Dimensions: ', imageData.GetDimensions())
print('Spacing: ', imageData.GetSpacing())
print('Origin: ', imageData.GetOrigin())

Dimensions:  (512, 512, 23)
Spacing:  (0.4688, 0.4688, 5.999993801116943)
Origin:  (-124.09700012207031, -140.11900329589844, -35.92409896850586)


# Custom Interactor Style

In [5]:
class SliceViewInteractorStyle(vtk.vtkInteractorStyleUser):
    def __init__(self, parent=None, imageViewer=None):
        self.AddObserver(vtkCommand.MouseWheelForwardEvent, self.mouseWheelForwardEvent)        
        self.AddObserver(vtkCommand.MouseWheelBackwardEvent, self.mouseWheelBackwardEvent)
        if imageViewer:
            self.imageViewer = imageViewer
            self.minSlice = imageViewer.GetSliceMin()
            self.maxSlice = imageViewer.GetSliceMax()
            self.slice = round((self.minSlice + self.maxSlice) / 2)
        
    def nextSlice(self):
        if self.imageViewer and self.slice < self.maxSlice:
            self.slice = self.slice + 1
            self.imageViewer.SetSlice(self.slice)

    def previousSlice(self):
        if self.imageViewer and self.slice > self.minSlice:
            self.slice = self.slice - 1
            self.imageViewer.SetSlice(self.slice)
    
    def mouseWheelForwardEvent(self, obj, event):
        self.nextSlice()
        return
    
    def mouseWheelBackwardEvent(self, obj, event):
        self.previousSlice()
        return

# Display VTK

Using itk python bindings seem to have issues and produce a noisy image

In [7]:
imageViewer = vtk.vtkImageViewer2()
imageViewer.SetInputData(imageData)
renderer = imageViewer.GetRenderer()

renderer.SetViewport((0,0,0.5,0.5))

renderer.GetActiveCamera().SetViewUp(0,-1,0) # itk origin is topleft, vtk origin is bottom left
renderer.ResetCamera()
renderWindow = imageViewer.GetRenderWindow() # Image viewer has its own render window !!!

interactor = vtk.vtkRenderWindowInteractor()
sliceInteractorStyle = SliceViewInteractorStyle(imageViewer=imageViewer)
interactor.SetInteractorStyle(sliceInteractorStyle)
interactor.SetRenderWindow(renderWindow)

renderWindow.SetSize(1000,1000)
renderWindow.Render()
interactor.Initialize()
interactor.Start()